# Preamble

Singapore is a country in Southeast Asia with a parliamentary democracy. Singapore has a single chamber legislature, the Singapore Parliament, which governs the country together with its president. Modelled after the Westminster system from the United Kingdom, the Singapore Parliament that includes elected Members of Parliament (MPs) as well as appointed Non-Constituency MPs and Nominated MPs.

# Aims

This exercise aims to:
1. Load data from the Hansard into sqlite for subsequent analysis
2. Carry out exploratory analysis including:
    1. Attendance records of MPs
    1. Questions raised including frequency and length of questions
    1. Analyse voting records
    1. Analyse adjournment motions
1. Carry out sentiment analysis of questions raised by MPs

# The Parliament of Singapore site

Official reports of Parliamentary debates are in the Hansard, and include attendance of Members, debate transcription and records of votes. All three will be examined in this exercise. The Hansard is published  within seven working days after each Parliament sitting.

To analyse

In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import datetime
import json
import sqlite3
from lxml import html

In [ ]:
PAGE = "https://www.parliament.gov.sg/parliamentary-business/votes-and-proceedings?parliament=14&fromDate=&toDate=&page=1&pageSize=100"

dates_source = requests.get(PAGE).text
date_pattern = r"Sitting on (\d{1,2}\s\w+\s\d{4})"
dates = re.findall(date_pattern, dates_source)
# dates

In [ ]:
dates

In [ ]:
dates_df = (pd.DataFrame(dates, columns=['sitting_dates'])
            .assign(sitting_date_dt_ts=lambda x: pd.to_datetime(x.sitting_dates),
                    sitting_date_dt_str=lambda x: [datetime.datetime.strftime(x, "%Y-%m-%d") for x in
                                                   x.sitting_date_dt_ts],
                    param_date=lambda x: [datetime.datetime.strftime(x, "%d-%m-%Y") for x in x.sitting_date_dt_ts]
                    )
            )

In [ ]:
# dates_df

In [ ]:
def load_raw_data():
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Origin': 'https://sprs.parl.gov.sg',
        'Referer': 'https://sprs.parl.gov.sg/search/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
    }

    json_data = {
        'headers': {
            'normalizedNames': {},
            'lazyUpdate': None,
        },
    }

    con = sqlite3.connect("/Users/whkoh/git-repos/sg-parl/data.db")
    dates_df1 = dates_df[:10]

    # for row in dates_df1.itertuples():
    #     print(row.param_date)

    for row in dates_df.itertuples():
        sitting_date_parsed = datetime.datetime.strptime(row.param_date, "%d-%m-%Y").strftime("%Y-%m-%d")
        params = {'sittingDate': row.param_date}
        print(row.param_date, sitting_date_parsed)
        response = requests.post('https://sprs.parl.gov.sg/search/getHansardReport/',
                                 params=params, headers=headers, json=json_data)
        raw_json = json.dumps(response.json())
        slim_json = re.sub(r"<img.*?>", "", raw_json)
        cur = con.cursor()
        cur.execute("""
               INSERT INTO json_raw
                 (sitting_date, raw)
               VALUES
                 (?, ?)
        """, (sitting_date_parsed, slim_json))
        con.commit()


In [ ]:
def test_db():
    f = "/Users/whkoh/git-repos/sg-parl/data.db.xz"
    import lzma
    import tempfile
    s = lzma.LZMAFile(f)
    tf = tempfile.NamedTemporaryFile()
    tf.write(s.read())
    con1 = sqlite3.connect(tf.name)
    cur1 = con1.cursor()
    sql = r"""SELECT t.* FROM json_raw t LIMIT 501;"""
    res = cur1.execute(sql)
    print(res.fetchone())

In [ ]:
def load_attendance():
    conn = sqlite3.connect("/Users/whkoh/git-repos/sg-parl/data.db")
    cur = conn.cursor()
    sql = """select sitting_date, json_extract(raw, '$.attendanceList') from json_raw t
    -- where sitting_date = '2022-10-20'
    ;"""
    r = cur.execute(sql)
    days = r.fetchall()
    overall_attendance_df = pd.DataFrame()
    member_constituency_df = pd.DataFrame()
    for day in days:
        attendance_df = (pd.DataFrame(json.loads(day[1]))
        .assign(sitting_date=pd.to_datetime(day[0], format="%Y-%m-%d"),
                ret=lambda x: [re.findall(r"(.*?)\s\((.*?)\)", y)[0] for y in x.mpName],
                member_name=lambda x: [y[0] for y in x.ret],
                constituency=lambda x: [y[1].split('(')[-1] for y in x.ret]
                )
        .rename(columns={'attendance': 'attended'})
        )
        row_mp_constituency = attendance_df[['member_name', 'constituency']]
        day_attendance = attendance_df[['member_name', 'attended', 'sitting_date']]
        overall_attendance_df = pd.concat([overall_attendance_df, day_attendance])
        member_constituency_df = pd.concat([row_mp_constituency, member_constituency_df])

    member_constituency_df.drop_duplicates(inplace=True)
    overall_attendance_df.to_sql('attendance', conn, if_exists='replace', index=False)
    member_constituency_df.to_sql('member', conn, if_exists='replace', index=False)
    conn.close()

load_attendance()

In [ ]:
df_attendance = (pd.DataFrame(response.json().get('attendanceList'))
                 .assign(ret=lambda x: [re.findall(r"(.*?)\s\((.*?)\)", y)[0] for y in x.mpName],
                         mp=lambda x: [y[0] for y in x.ret],
                         constituency=lambda x: [y[1] for y in x.ret]
                         )
                 )

In [ ]:
df_attendance.head(30)

In [ ]:
def load_section():
    conn = sqlite3.connect("/Users/whkoh/git-repos/sg-parl/data.db")
    cur = conn.cursor()
    sql = """select sitting_date, json_extract(raw, '$.takesSectionVOList') from json_raw t
    where sitting_date = '2022-10-05'
    ;"""
    r = cur.execute(sql)
    i, j = r.fetchone()
    df = (pd.DataFrame.from_records(json.loads(j))
          .assign(sitting_date=i)
          .astype(str)
          .reset_index()
          .rename(columns={'index': 'item_nr'})
          .to_sql('item', con=conn, if_exists='append', index=False)
          )

    return

load_section()


In [ ]:
translator = str.maketrans({chr(10): '', chr(9): '', ':': '', '[': '', ']': ''})

def load_item_speech():
    conn = sqlite3.connect("/Users/whkoh/git-repos/sg-parl/data.db")
    cur = conn.cursor()
    sql = """select sitting_date, title, item_nr, content, sectionType, reportType from item t
    where item_nr in ('16', '17', '18', '19')
    ;"""
    r = cur.execute(sql)
    rfa = r.fetchall()
    for rf in rfa:
        try:
            if rf[4] == 'OS' and rf[5] == 'Matter Raised On Adjournment Motion':
                tree = html.fromstring(rf[3])
                _d = [(l.xpath('strong/text()'), l.text) for l in tree] # Use fillna and down to get a df
                filled_df = pd.DataFrame(_d, columns=['mp', 'text'])
                filled_df = (filled_df
                             .mask(filled_df.applymap(str).eq('[]'))
                             .fillna(method='ffill', axis=0)
                             .assign(mp = lambda x: [y[0] for y in x.mp],
                                     sitting_date = rf[0],
                                     item = rf[2]
                                     )
                             )

            elif rf[4] == 'WANA' or rf[4] == 'WA' or rf[4] == 'OA' or rf[4] == 'OS':
                tree = html.fromstring(rf[3])
                _d = [(l.xpath('strong/text()'), l.xpath('text()')) for l in tree] #
                filled_df = pd.DataFrame(_d, columns=['mp', 'text'])
                filled_df = (filled_df
                             .mask(filled_df.applymap(str).eq('[]'))
                             .fillna(method='ffill', axis=0)
                             .dropna()
                             .assign(mp = lambda x: [y[0].translate(translator) for y in x.mp],
                                     text = lambda x: [' '.join(y).translate(translator) for y in x.text],
                                     sitting_date = rf[0],
                                     item = rf[2]
                                     )
                             )
            # elif rf[4] == 'OS':
            #     tree = html.fromstring(rf[3])
            #     _d = [(l.xpath('strong/text()'), l.xpath('text()')) for l in tree] #
            #     filled_df = pd.DataFrame(_d, columns=['mp', 'text'])
            #     filled_df = (filled_df
            #                  .mask(filled_df.applymap(str).eq('[]'))
            #                  .fillna(method='ffill', axis=0)
            #     .dropna()
            #                  .assign(mp = lambda x: [y[0] for y in x.mp],
            #                  #         text = lambda x: [' '.join(y).translate(translator) for y in x.text],
            #                  #         sitting_date = rf[0],
            #                  #         item = rf[2]
            #                          )
            #                  )
            #
            #     return filled_df
            else:
                print(rf[4], rf[5], rf[0], rf[2], "Not implemented")
                raise NotImplementedError
            filled_df.to_sql('speech', conn, if_exists='append', index=False)
        except Exception:
            print(rf[4], rf[5], rf[0], rf[2], "Exception")
    return

df2 = load_item_speech()
# type(df2.iloc[0,1][0])

In [ ]:
df2